### 01. High Price Predict Model with Linear Regression

In [1]:
# 시간 측정 시작
import time
start = time.time()

# 라이브러리
import pandas as pd
kospi = pd.read_csv('./KOSPI.csv')

# KOSPI에 상장된 종목 코드
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))
code_list = kospi['종목코드'].unique()

# 라이브러리
import numpy as np

# 진행과정 Check
check = 0

# 결과 저장 데이터프레임
result_df = pd.DataFrame()

# 코드 별 진행
for code in code_list[:] : 

    # Hyper Parameter
    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   # 학습 기간 시작점
    end_date = '20200229'     # 학습 기간 종료점
    Lookup = '9999' 
    
    # url
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    # url open
    from urllib.request import urlopen
    url_page = urlopen(url)

    # json 파일로 받아오기
    import json
    url_data = json.loads(url_page.read())

    # json을 데이터프레임으로 바꾸기
    import pandas as pd
    data = pd.DataFrame(url_data)
    
    # Data Column명 변경
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
    # 순서 뒤집기
    data = data[::-1]
    data = data.set_index('date')
    
    # 필요한 변수만 추출
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    # 만약에 데이터가 비어있다면 그 종목은 PASS
    if data["open"][0] == '' : 
        continue
    
    # 변수 속성 정수로 바꾸기
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
    # 거래량과 거래가격을 모두 로그 변환
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    # 각각의 피쳐들에 대해 
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    
    # 타겟 생성 : 다음날의 고가를 예측
    data['y'] = data['high'].shift(-1)
    
    # NaN이 들어가 있는 행 제거
    data = data.dropna()
    
    # Input과 Target을 분리
    X = data[data.columns[:-1]]
    y = data['y']
    
    # Linear Regression
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    # Test Data 생성하기
    start_date, end_date = '20200301', '20200319'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    url_page = urlopen(url)
    url_data = json.loads(url_page.read())
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-1)
        
    data = data[-1:]
    
    data = data[data.columns[:-1]]

    # 필요한것들
    result = dict({'종목' : code,
                   '어제 날짜' : data.index.values[0],
                   '어제 종가' : int(data['close'].values),
                   '오늘 고가' : int(round(reg.predict(data)[0])),
                   '오늘 고가 변동폭 예측' : 100*(int(round(reg.predict(data)[0]))-int(data['close'].values))/\
                                     int(data['close'].values)})
    result_df = result_df.append(result, ignore_index=True)
    
    # 각 종목별 결과 출력
    if check % 100 == 0 : 
        print('{:4} / {:4} 종목 ======> {:7.2f}% 완료'.format(check, 2144, 100*check/2144))
    check += 1

# 전체 완료
print('{:4} / {:4} 종목 ======> {:7.2f}% 완료'.format(check + 1, 2144, 100*check/2144))

# 시간측정 완료
print("\n\ntime :", time.time() - start)

   0 / 2144 종목 ======>    0.00% 완료
 100 / 2144 종목 ======>    4.66% 완료
 200 / 2144 종목 ======>    9.33% 완료
 300 / 2144 종목 ======>   13.99% 완료
 400 / 2144 종목 ======>   18.66% 완료
 500 / 2144 종목 ======>   23.32% 완료
 600 / 2144 종목 ======>   27.99% 완료
 700 / 2144 종목 ======>   32.65% 완료
 800 / 2144 종목 ======>   37.31% 완료
 900 / 2144 종목 ======>   41.98% 완료
1000 / 2144 종목 ======>   46.64% 완료
1100 / 2144 종목 ======>   51.31% 완료
1200 / 2144 종목 ======>   55.97% 완료
1300 / 2144 종목 ======>   60.63% 완료
1400 / 2144 종목 ======>   65.30% 완료
1500 / 2144 종목 ======>   69.96% 완료
1600 / 2144 종목 ======>   74.63% 완료
1700 / 2144 종목 ======>   79.29% 완료
1800 / 2144 종목 ======>   83.96% 완료
1900 / 2144 종목 ======>   88.62% 완료
2000 / 2144 종목 ======>   93.28% 완료
2100 / 2144 종목 ======>   97.95% 완료
2145 / 2144 종목 ======>  100.00% 완료


### 02. 예측 결과

In [2]:
result_df = result_df[['어제 날짜', '종목', '어제 종가', '오늘 고가', '오늘 고가 변동폭 예측']]

In [3]:
result_df['오늘 고가 변동폭 예측'] = result_df['오늘 고가 변동폭 예측'].apply(lambda x : round(x,2))

In [4]:
result_df

,어제 날짜,종목,어제 종가,오늘 고가,오늘 고가 변동폭 예측
0,20200318,060310,1705.0,1812.0,6.28
1,20200318,095570,2895.0,2810.0,-2.94
2,20200318,068400,6840.0,7092.0,3.68
3,20200318,006840,15450.0,15233.0,-1.40
4,20200318,054620,5980.0,6197.0,3.63
...,...,...,...,...,...
2139,20200318,189980,986.0,1023.0,3.75
2140,20200318,000540,1730.0,2009.0,16.13
2141,20200318,003280,229.0,243.0,6.11
2142,20200318,037440,3700.0,3753.0,1.43


### 03. 내일 고가 변동폭이 가장 높은 TOP 10

In [5]:
result_df = result_df.sort_values(by='오늘 고가 변동폭 예측', ascending=False).set_index('종목')

In [6]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] < 30.00]

In [7]:
result_df[:10]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
307930,20200318,5770.0,7480.0,29.64
015260,20200318,199.0,256.0,28.64
203650,20200318,1400.0,1788.0,27.71
226350,20200318,298.0,379.0,27.18
057030,20200318,4385.0,5514.0,25.75
066430,20200318,1010.0,1268.0,25.54
251630,20200318,5060.0,6335.0,25.20
019770,20200318,1540.0,1925.0,25.00
021080,20200318,1205.0,1502.0,24.65


### 04. 특정 종목 검색

In [8]:
# result_df.loc['000000']